In [ ]:
!pip install mysql-connector-python

In [ ]:
import pandas as pd
import mysql.connector

# 1. CSV 파일 불러오기
csv_path = "./sencpick_product_data.csv"
df = pd.read_csv(csv_path)

# NaN 처리 (옵션: 빈 문자열이나 None으로 변환)
# df = df.where(pd.notnull(df), None)

# 2. MySQL 연결
conn = mysql.connector.connect(
    host="localhost",
    user="root",        # ← 사용자 이름 입력
    password="root",# ← 비밀번호 입력
    database="senpick_db",      # ← 사용할 DB 이름 입력
)
cursor = conn.cursor()

# 3. INSERT SQL 구문 (컬럼명은 CSV/테이블 구조에 맞게 수정)
insert_sql = """
INSERT INTO PRODUCT (
    name, brand, category, sub_category, options, price, link, thumbnail_url
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# 4. 데이터 삽입
for _, row in df.iterrows():
    cursor.execute(insert_sql, (
        row["product_name"],
        row.get("brand", "브랜드 없음"),
        row["category_main"],
        row["category_sub"],
        row["options"][:255],
        float(row["price"]),
        row["product_url"][:255],
        row.get("thumbnail_url", "썸네일 없음"),
    ))

# 5. 커밋 및 종료
conn.commit()
cursor.close()
conn.close()

print("✅ CSV 데이터가 MySQL에 성공적으로 삽입되었습니다.")

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client.models import Filter, FieldCondition, Range, MatchValue
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings

qdrant = QdrantClient(url="http://localhost:6333", prefer_grpc=False)
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = QdrantVectorStore(
    client=qdrant,
    collection_name="senpick",
    embedding=embedding,
)

In [ ]:
qdrant.scroll(
    collection_name="senpick",
    scroll_filter=Filter(
        must=[FieldCondition(key="price", range=Range(gte=10000, lte=50000))]
    ),
    limit=3
)

In [ ]:
docs = vectorstore.similarity_search(
            "고마운 멘토에게 5만원대의 멋진 스타일 선물을 준비했는데", 
            k=5,
            filter=Filter(
                must=[
                    FieldCondition(
                        key="title", 
                        match=MatchValue(value="[아트박스] 5만원권")
                    )
                ]
            )
)
docs

In [ ]:
from qdrant_client.models import Filter, FieldCondition, Range, SearchRequest

query_vector = embedding.embed_query("고마운 멘토에게 5만원대의 멋진 스타일 선물을 준비했는데")

results = qdrant.search(
    collection_name="senpick",
    query_vector=query_vector,
    limit=5,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="price",
                range=Range(gte=10000, lte=50000)
            )
        ]
    )
)
results

In [ ]:
vectorstore.similarity_search(
            "고마운 멘토에게 5만원대의 멋진 스타일 선물을 준비했는데", 
            k=20,
            filter=Filter(
                must=[
                    FieldCondition(
                        key="Price",
                        range=Range(
                            gte=9999,
                            # lte=50000,
                        ),
                    )
                ],
            )
        )